In [1]:
import sys, os

In [2]:
import nmslib

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [3]:
import numpy as np
from tqdm import tqdm

In [4]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [5]:
!pwd

/home/yblainm/projects/retrievalogy/notebooks


In [5]:
sys.path.append('..')

In [6]:
from src.datasets.tatoeba import TatoebaEN

DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
/home/yblainm/projects/retrievalogy/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data_fasttext = TatoebaEN(vectorization_method="fasttext")

INFO:nmslib:Loading index from /home/yblainm/projects/retrievalogy/data/eng_sentences_fasttext.nms
INFO:nmslib:Loading optimized index.
INFO:nmslib:searchMethod: 3
INFO:nmslib:Total: 1425903, Memory per object: 1348
INFO:nmslib:Finished loading index
INFO:nmslib:Set HNSW query-time parameters:
INFO:nmslib:ef(Search)         =20
INFO:nmslib:algoType           =2


In [8]:
data_fasttext.index.knnQueryBatch([data_fasttext.vectors[0]])

[(array([     0,  59505,   1442,  81265,   1363,  59614,   1423, 166102,
          81493,  11748], dtype=int32),
  array([5.9604645e-08, 3.0917227e-02, 3.0994177e-02, 4.2958915e-02,
         6.1503649e-02, 6.7718565e-02, 6.7822933e-02, 6.9813490e-02,
         7.0812106e-02, 7.0814312e-02], dtype=float32))]

In [7]:
from datasets import load_metric

In [8]:
edit_dist = load_metric("ter")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/ter/ter.py HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/2.2.2/metrics/ter/ter.py HTTP/1.1" 200 0


In [18]:
def edit_distance(analogies):
    """
    Input is a list of 4-tuples of strings (a,b,c,d)
    """
    def getdist(x,y):
        return edit_dist.compute(
            predictions=[x],
            references=[[y]],
            case_sensitive=True
        )["num_edits"]
          
    edit_dists = np.array(#torch.tensor(
        tuple(
            tuple(getdist(x,y) for x,y in ((anlg[0],anlg[1]),(anlg[0], anlg[2]),(anlg[1],anlg[3]),(anlg[2],anlg[3])))
            for anlg in analogies#tqdm(analogies)
        ),
        dtype=float
    )
    # ["num_edits"] for sequence_pred, sequence_ref in zip(predictions, references)], dtype=torch.float)
    mean = edit_dists.mean(axis=0)
    std = edit_dists.std(axis=0)#dim=-1)
    return mean, std
                      
# {'score': 40.0, 'num_edits': 2, 'ref_length': 5.0}

In [12]:
edit_distance([("hey I'm yves", "hey you're yves", "hey you're troy", "hey he's troy"),
               ("hey how's yves", "hey it's yves", "wow is that yves", "wow you're yves")])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 37.78it/s]


(array([1. , 2.5, 2. , 1.5]), array([0. , 0.5, 0. , 0.5]))

In [13]:
ia = 101 # 456783
a = data_fasttext.vectors[ia]
sa = data_fasttext.sentences[ia]
sa

'We often go there.'

In [14]:
ib = data_fasttext.index.knnQuery(a)[0][1]#[5]
b = data_fasttext.vectors[ib]
sb = data_fasttext.sentences[ib]
sb

'We go there frequently.'

In [15]:
ic = data_fasttext.index.knnQuery(b)[0][-1]#[4]
c = data_fasttext.vectors[ic]
sc = data_fasttext.sentences[ic]
sc

"We don't have to go there today."

In [16]:
d = c + b - a
i_d = data_fasttext.index.knnQuery(d)[0][4]#[4]
sd = data_fasttext.sentences[i_d]
sd

"You don't have to go there today."

In [17]:
i_knn = data_fasttext.index.knnQuery(a)[0]
i_knn[~(i_knn == np.array([ia, ib, ic, i_d])[...,None]).any(axis=0)][3]

31438

In [ ]:
data_fasttext.index.knnQueryBatch(data_fasttext.vectors[:200000], num_threads=4)

In [13]:
def get_analogy(data: TatoebaEN, ia: int):
    
    def dedup(i_knn, *idx):
        return i_knn[~(i_knn == np.array(idx)[...,None]).any(axis=0)]
    
    a = data.vectors[ia]
    sa = data.sentences[ia]
    
    ib = dedup(data.index.knnQuery(a)[0], ia)[3]
    b = data.vectors[ib]
    sb = data.sentences[ib]
    
    ic = dedup(data.index.knnQuery(b, k=15)[0], ia, ib)[-1]
    c = data.vectors[ic]
    sc = data.sentences[ic]
    
    d = c + b - a
    i_d = dedup(data.index.knnQuery(d)[0], ia, ib, ic)[3]
    sd = data.sentences[i_d]
    
    return (ia, ib, ic, i_d), (sa, sb, sc, sd), (a, b, c, d)

In [19]:
get_analogy(data_fasttext, 10012)[:-1]

((10012, 34922, 48324, 222431),
 ('Tom is stark naked.',
  'Tom is intimidating.',
  'Tom is enthusiastic.',
  'Tom is very intimidating.'))

In [29]:
i_anlg, s_anlg, v_anlg = zip(*tuple(get_analogy(data_fasttext, ia) for ia in tqdm(range(100000))))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:39<00:00, 2563.46it/s]


In [30]:
# i_anlg, s_anlg

In [31]:
mean, std = edit_distance(s_anlg)
mean, std

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [15:43<00:00, 105.94it/s]


In [33]:
mean, std

(array([2.01169, 3.25917, 3.17746, 2.57774]),
 array([1.39836095, 1.96181572, 2.16225992, 2.01190867]))

In [39]:
f"{3.5124123:0.2f}"

'3.51'

In [43]:
print('\t'.join(f"{m:0.2f}+-{s:0.2f}" for m,s in zip(mean, std)))

2.01+-1.40	3.26+-1.96	3.18+-2.16	2.58+-2.01


In [51]:
print('\t'.join(s_anlg[0]))

Ziri is a foreman.	Ziri is a janitor.	Ziri is a dermatologist.	Ziri is a biologist.


In [53]:
with open("../results/lsh_experiments_fasttext.tsv", 'w') as f:
    f.write("a_b\ta_c\tb_d\tc_d\n")
    f.write('\t'.join(f"{m:0.2f}+-{s:0.2f}" for m,s in zip(mean, std)))
    f.write('\n')
    f.write("a\tb\tc\td\n")
    f.writelines('\t'.join(abcd) + '\n' for abcd in s_anlg)

In [11]:
data_sbert = TatoebaEN(vectorization_method="transformer")

INFO:nmslib:Loading index from /home/yblainm/projects/retrievalogy/data/eng_sentences_transformer.nms
INFO:nmslib:Loading optimized index.
INFO:nmslib:searchMethod: 3
INFO:nmslib:Total: 1425903, Memory per object: 3220
INFO:nmslib:Finished loading index
INFO:nmslib:Set HNSW query-time parameters:
INFO:nmslib:ef(Search)         =20
INFO:nmslib:algoType           =2


In [ ]:
i_anlg, s_anlg, v_anlg = zip(*tuple(get_analogy(data_sbert, ia) for ia in tqdm(range(10000))))

In [16]:
s_anlg[3:5]

(("We're guests here.",
  "You're all our guests tonight.",
  "We're having five guests tonight.",
  "Tomorrow evening we'll have five guests."),
 ('How did you sleep?',
  'Did you sleep well?',
  'Did you have a good night?',
  'Did you have a good sleep?'))

In [19]:
mean, std = edit_distance(s_anlg)
mean, std

(array([3.6372, 4.8096, 4.6617, 4.3552]),
 array([1.8512634 , 2.0155763 , 2.18514373, 2.25597716]))

In [20]:
with open("../results/lsh_experiments_sbert.tsv", 'w') as f:
    f.write("a_b\ta_c\tb_d\tc_d\n")
    f.write('\t'.join(f"{m:0.2f}+-{s:0.2f}" for m,s in zip(mean, std)))
    f.write('\n')
    f.write("a\tb\tc\td\n")
    f.writelines('\t'.join(abcd) + '\n' for abcd in s_anlg)

In [21]:
print('\t'.join(f"{m:0.2f}+-{s:0.2f}" for m,s in zip(mean, std)))

3.64+-1.85	4.81+-2.02	4.66+-2.19	4.36+-2.26
